In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
from tqdm import tqdm

# Custom Dataset class
class CustomImageDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        
        # Get all image files
        self.image_files = sorted([f for f in os.listdir(images_dir) 
                                   if f.endswith(('.png', '.jpg', '.jpeg', '.JPG', '.PNG'))])
        
        # Read all labels and create class mapping
        self.labels = []
        all_classes = set()
        
        for img_file in self.image_files:
            # Assume label file has same name but .txt extension
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(labels_dir, label_file)
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    # Read first line and extract class (assumes YOLO format: class x y w h)
                    line = f.readline().strip()
                    if line:
                        class_id = int(line.split()[0])
                        self.labels.append(class_id)
                        all_classes.add(class_id)
                    else:
                        self.labels.append(0)  # Default class if empty
            else:
                self.labels.append(0)  # Default class if no label file
        
        # Create sorted list of classes
        self.classes = sorted(list(all_classes))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Remap labels to consecutive indices
        self.labels = [self.class_to_idx[label] for label in self.labels]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Get label
        label = self.labels[idx]
        
        return image, label

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Dataset paths
train_images_dir = "Week9 - object detection/data/train/images"
train_labels_dir = "Week9 - object detection/data/train/labels"
test_images_dir = "Week9 - object detection/data/test/images"
test_labels_dir = "Week9 - object detection/data/test/labels"

# Create datasets
train_data = CustomImageDataset(train_images_dir, train_labels_dir, transform=train_transform)
test_data = CustomImageDataset(test_images_dir, test_labels_dir, transform=test_transform)

# Create dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Number of classes: {len(train_data.classes)}")
print(f"Classes: {train_data.classes}")

# Model setup
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
Learning_Rate = 0.001  # Define if not already defined
optimizer = optim.Adam(model.parameters(), lr=Learning_Rate)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# Save model
torch.save(model.state_dict(), "resnet18_finetuned.pth")
print("Model saved successfully!")

FileNotFoundError: [Errno 2] No such file or directory: 'Week9 - object detection/data/train/images'